In [17]:
import requests

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

data = requests.get(standings_url)

from bs4 import BeautifulSoup

soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]

team_urls = [f"https://fbref.com{l}" for l in links]

data = requests.get(team_urls[0])

import pandas as pd
matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

data = requests.get(f"https://fbref.com{links[0]}")

shooting = pd.read_html(data.text, match="Shooting")[0]

shooting.head()

/var/folders/5d/rtn2kns94l59c45vj5n555580000gn/T/ipykernel_11863/2452401508.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/5d/rtn2kns94l59c45vj5n555580000gn/T/ipykernel_11863/2452401508.py:29: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


For Arsenal                                                              \
         Date   Time              Comp                Round  Day    Venue   
0  2023-08-06  16:00  Community Shield  FA Community Shield  Sun  Neutral   
1  2023-08-12  12:30    Premier League          Matchweek 1  Sat     Home   
2  2023-08-21  20:00    Premier League          Matchweek 2  Mon     Away   
3  2023-08-26  15:00    Premier League          Matchweek 3  Sat     Home   
4  2023-09-03  16:30    Premier League          Matchweek 4  Sun     Home   

                                         ... Standard               Expected  \
  Result     GF     GA         Opponent  ...     Dist   FK PK PKatt       xG   
0      D  1 (4)  1 (1)  Manchester City  ...      NaN  NaN  0     0      NaN   
1      W      2      1  Nott'ham Forest  ...     19.1  0.0  0     0      0.8   
2      W      1      0   Crystal Palace  ...     16.4  0.0  1     1      2.0   
3      D      2      2           Fulham  ...     13.8  0.0  1     1      3.2   
4      W      3      1   Manchester Utd  ...     15.0  0.0  0     0      2.3   

                            Unnamed: 25_level_0  
  npxG npxG/Sh G-xG np:G-xG        Match Report  
0  NaN     NaN  NaN     NaN        Match Report  
1  0.8    0.06  1.2     1.2        Match Report  
2  1.2    0.09 -1.0    -1.2        Match Report  
3  2.4    0.14 -1.2    -1.4        Match Report  
4  2.3    0.13  0.7     0.7        Match Report  

[5 rows x 26 columns]

In [18]:
years = list(range(2023,2019, -1))
all_matches = []
standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

import time
for year in years:
    data = requests.get(standings_url)
    soup = BeautifulSoup(data.text)
    standings_table = soup.select('table.stats_table')[0]

    links = [l.get("href") for l in standings_table.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"https://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standings_url = f"https://fbref.com{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-Stats", "").replace("-", " ")
        data = requests.get(team_url)
        matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
        time.sleep(1)
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = requests.get(f"https://fbref.com{links[0]}")
        shooting = pd.read_html(data.text, match="Shooting")[0]
        time.sleep(1)
        shooting.columns = shooting.columns.droplevel()
        try:
            team_data = matches.merge(shooting[["Date", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="Date")
        except ValueError:
            continue
        team_data = team_data[team_data["Comp"] == "Premier League"]
        
        team_data["Season"] = year
        team_data["Team"] = team_name
        all_matches.append(team_data)
        time.sleep(1)

/var/folders/5d/rtn2kns94l59c45vj5n555580000gn/T/ipykernel_11863/3865190277.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/5d/rtn2kns94l59c45vj5n555580000gn/T/ipykernel_11863/3865190277.py:27: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]
/var/folders/5d/rtn2kns94l59c45vj5n555580000gn/T/ipykernel_11863/3865190277.py:21: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
/var/folders/5d/rtn2kns94l59c45vj5n555580000gn/T/ipyke

In [19]:
len(all_matches)

80

In [20]:
match_df = pd.concat(all_matches)

In [21]:
match_df.columns = [c.lower() for c in match_df.columns]

In [22]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
1,2023-08-12,12:30,Premier League,Matchweek 1,Sat,Home,W,2,1,Nott'ham Forest,...,Match Report,NaN,15.0,7.0,19.1,0.0,0,0,2023,Arsenal
2,2023-08-21,20:00,Premier League,Matchweek 2,Mon,Away,W,1,0,Crystal Palace,...,Match Report,NaN,13.0,2.0,16.4,0.0,1,1,2023,Arsenal
3,2023-08-26,15:00,Premier League,Matchweek 3,Sat,Home,D,2,2,Fulham,...,Match Report,NaN,18.0,9.0,13.8,0.0,1,1,2023,Arsenal
4,2023-09-03,16:30,Premier League,Matchweek 4,Sun,Home,W,3,1,Manchester Utd,...,Match Report,NaN,17.0,5.0,15.0,0.0,0,0,2023,Arsenal
5,2023-09-17,16:30,Premier League,Matchweek 5,Sun,Away,W,1,0,Everton,...,Match Report,NaN,13.0,4.0,17.4,0.0,0,0,2023,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,2021-05-02,19:15,Premier League,Matchweek 34,Sun,Away,L,0,4,Tottenham,...,Match Report,NaN,8.0,1.0,18.2,0.0,0,0,2020,Sheffield United
39,2021-05-08,15:00,Premier League,Matchweek 35,Sat,Home,L,0,2,Crystal Palace,...,Match Report,NaN,7.0,0.0,13.4,1.0,0,0,2020,Sheffield United
40,2021-05-16,19:00,Premier League,Matchweek 36,Sun,Away,W,1,0,Everton,...,Match Report,NaN,10.0,3.0,18.5,0.0,0,0,2020,Sheffield United
41,2021-05-19,18:00,Premier League,Matchweek 37,Wed,Away,L,0,1,Newcastle Utd,...,Match Report,NaN,11.0,1.0,18.3,1.0,0,0,2020,Sheffield United


In [23]:
match_df.to_csv("matches.csv")